In [1]:
import constants as c
import pandas as pd
import glob

from wikidata.client import Client

import api.hf_api_calls as hf


In [2]:
ROOT = "KGData/*"
folders = glob.glob(ROOT)

files = []

for folder in folders:
    files.append(glob.glob(f'{folder}/*.csv'))

In [3]:
files

[['KGData/yago15k/yago15k_train.csv',
  'KGData/yago15k/yago15k_valid.csv',
  'KGData/yago15k/yago15k_test.csv'],
 [],
 ['KGData/wikidata/wiki_valid.csv',
  'KGData/wikidata/wiki_train.csv',
  'KGData/wikidata/wiki_test.csv']]

In [11]:
df = pd.read_csv(f"{c.WIKIDATA_PATH}wiki_train.csv",
                 names=["subject", "predicate", "object", "time_predicate", "time_stamp", "question", hf.T0PP])

In [12]:
df

,subject,predicate,object,time_predicate,time_stamp,question,bigscience/T0pp
0,Q215841,P54,Q18744,occurUntil,2015,NaN,NaN
1,Q524660,P1082,1350,occurUntil,1925,NaN,NaN
2,Q330588,P1082,118,occurUntil,1900,NaN,NaN
3,Q285107,P1082,491,occurUntil,1930,NaN,NaN
4,Q696162,P54,Q392310,occurSince,2003,NaN,NaN
...,...,...,...,...,...,...,...
121417,Q202329,P54,Q132885,occurUntil,2013,NaN,NaN
121418,Q81051,P1082,2341,occurSince,1966,NaN,NaN
121419,Q90489,P1087,2588,occurSince,2015,NaN,NaN
121420,Q379462,P1087,2409,occurUntil,2005,NaN,NaN


In [13]:
client = Client()
def translate_entry(entry):
    result = []
    for value in entry:
        if isinstance(value, str):
            if value.startswith('Q') or value.startswith('P'):
                en_label = client.get(value, load=True).label.get('en')
                result.append(en_label)
                continue
        result.append(value)
    return result

In [14]:
def make_question(entry):
    subject = entry[0]
    predicate = entry[1]
    object = entry[2]
    # question_start = entry[3][5:] + ' when is'
    question_start = 'When was'
    question = f'{question_start} {subject} {predicate} {object}?'
    return question

In [18]:
for i, values in df[0:1].iterrows():
    entry = values.tolist()
    plain_entry = translate_entry(entry)
    question = make_question(plain_entry)
    df.loc[i, 'question'] = question
    df.loc[i, hf.T0PP] = hf.query(question, hf.T0PP)[0]['generated_text']

In [22]:
def fetch_answers(start, stop):
    try:
        for i, values in df[start:stop].iterrows():
            entry = values.tolist()
            plain_entry = translate_entry(entry)
            question = make_question(plain_entry)
            df.loc[i, 'question'] = question
            df.loc[i, hf.T0PP] = hf.query(question, hf.T0PP)[0]['generated_text']
        df.to_csv(f"{c.WIKIDATA_PATH}wiki_train_t0pp.csv")
    except:
        df.to_csv(f"{c.WIKIDATA_PATH}wiki_train_t0pp.csv")

In [ ]:
fetch_answers(0, 500)